In [159]:
from functools import reduce
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import os
import seaborn as sns
import datetime

In [160]:
data=pd.read_csv('train (2).csv')
sample=pd.read_csv('sample_sub.csv')
client_data=pd.read_csv('client_data.csv')
payement_hist=pd.read_csv('payment_history.csv')
policy_data=pd.read_csv('policy_data.csv')

In [161]:
data.head(2)

,Policy ID,Lapse,Lapse Year
0,PID_4928TWH,?,?
1,PID_KBLLEGK,?,?


In [162]:
data['Lapse'] =data['Lapse'].replace({'?':0, '1':1});data['Lapse']=data['Lapse'].astype(int)

In [163]:
data.dtypes

Policy ID     object
Lapse          int32
Lapse Year    object
dtype: object

In [164]:
test =data[data['Lapse'] ==  0].reset_index(drop=True)
train =data[data['Lapse'] ==  1].reset_index(drop=True)

In [165]:
print(train['Lapse Year'].unique(),test['Lapse Year'].unique() )

['2019' '2018' '2017'] ['?']


In [166]:
train['Lapse Year'].unique()

array(['2019', '2018', '2017'], dtype=object)

In [167]:
print(train.shape, test.shape, sample.shape)

(7978, 3) (43707, 3) (43707, 2)


In [168]:
client_data.head(2)

,Policy ID,NPH_TITLE,NPH_LASTNAME,NPH_SEX,NPH_BIRTHDATE,NAD_ADDRESS1,NAD_ADDRESS2
0,PID_P1UHZRT,Ms.,NaN,F,1997,ADDRESS1_WSRXQ43,ADDRESS2_AIIHWOT
1,PID_GUI5GQJ,Mr.,NPH_LASTNAME_L256UFJ,M,1954,ADDRESS1_WSRXQ43,ADDRESS2_AIIHWOT


In [169]:
payement_hist.head()

,Policy ID,AMOUNTPAID,DATEPAID,POSTDATE,PREMIUMDUEDATE
0,PID_VTBGKIZ,1287.423813,2018-12-29 00:00:00,2018-12-29 00:00:00,2018-01-11 00:00:00
1,PID_9J9V5YQ,0.000000,2017-04-10 00:00:00,2017-04-10 17:30:00,2017-01-10 00:00:00
2,PID_RT2SI59,3435.950888,2018-12-24 00:00:00,2018-12-28 00:00:00,2019-01-01 00:00:00
3,PID_NCXO0DU,3196.364996,2018-06-26 00:00:00,2018-06-26 00:00:00,2018-01-07 00:00:00
4,PID_FZYCT7V,0.000000,2018-03-12 00:00:00,2018-03-12 17:09:07,2018-01-12 00:00:00


In [170]:
payement_hist['AMOUNTPAID'].unique()

array([ 1287.42381255,     0.        ,  3435.95088835, ...,
       10869.5850967 ,  6685.47922141,  9113.29765579])

In [171]:
data_frames_train =[train,client_data,policy_data,payement_hist]
data_frames_test =[test,client_data,policy_data,payement_hist]

In [172]:
train = reduce(lambda  left,right: pd.merge(left,right,on=['Policy ID'],
                                            how='outer'), data_frames_train)
test = reduce(lambda  left,right: pd.merge(left,right,on=['Policy ID'],
                                            how='outer'), data_frames_test)

In [173]:
print(train.shape, test.shape)

(5437264, 26) (5437264, 26)


In [174]:
cols_to_drop=['NPH_TITLE','NPH_LASTNAME_x','Lapse Year','NPH_BIRTHDATE','NAD_ADDRESS2','NAD_ADDRESS1','NPH_LASTNAME_y','PCL_LOCATCODE','NPH_SEX','AAG_AGCODE','CATEGORY','POSTDATE']

In [175]:
train=train.drop(cols_to_drop, axis=1)
test=test.drop(cols_to_drop, axis=1)#,'NPH_SEX'

In [176]:
train.isna().sum()

Policy ID                0
Lapse              4614085
NP2_EFFECTDATE           0
PPR_PRODCD               0
NPR_PREMIUM            744
CLF_LIFECD               0
NSP_SUBPROPOSAL          0
NPR_SUMASSURED     1571050
NLO_TYPE                 0
NLO_AMOUNT         3463038
OCCUPATION               0
AMOUNTPAID          456214
DATEPAID            399322
PREMIUMDUEDATE      522451
dtype: int64

In [177]:
train = train.drop_duplicates(subset='Policy ID', keep="first")
test = test.drop_duplicates(subset='Policy ID', keep="first")

In [178]:
train=train.reset_index(drop=True)
test=test.reset_index(drop=True)

In [179]:
print(test.shape, train.shape)

(51685, 14) (51685, 14)


In [180]:
train.head(2)

,Policy ID,Lapse,NP2_EFFECTDATE,PPR_PRODCD,NPR_PREMIUM,CLF_LIFECD,NSP_SUBPROPOSAL,NPR_SUMASSURED,NLO_TYPE,NLO_AMOUNT,OCCUPATION,AMOUNTPAID,DATEPAID,PREMIUMDUEDATE
0,PID_MFAAYNJ,1.0,1/10/2018,PPR_PRODCD_KOFUYNN,1074.485839,4,333,27631.086529,NLO_TYPE_DPBHSAH,NaN,OCCUPATION_NNHJ7XV,0.0,2018-01-10 00:00:00,2018-01-10 00:00:00
1,PID_TICDPAY,1.0,1/4/2017,PPR_PRODCD_165U4LY,1243.869117,1,111,145010.003453,NLO_TYPE_XAJI0Y6,215.354032,OCCUPATION_NNHJ7XV,NaN,NaN,NaN


In [181]:
sample.tail()

,Policy ID,Lapse
43702,PID_FSR6DS3,0
43703,PID_98JLBWI,0
43704,PID_HM8H99C,0
43705,PID_KQH5U5I,0
43706,PID_HWFS2BA,0


In [182]:
test=test[:43707]

In [183]:
#complete query

In [184]:
print(test.shape, train.shape)

(43707, 14) (51685, 14)


In [185]:
train.columns

Index(['Policy ID', 'Lapse', 'NP2_EFFECTDATE', 'PPR_PRODCD', 'NPR_PREMIUM',
       'CLF_LIFECD', 'NSP_SUBPROPOSAL', 'NPR_SUMASSURED', 'NLO_TYPE',
       'NLO_AMOUNT', 'OCCUPATION', 'AMOUNTPAID', 'DATEPAID', 'PREMIUMDUEDATE'],
      dtype='object')

In [186]:
train.dtypes

Policy ID           object
Lapse              float64
NP2_EFFECTDATE      object
PPR_PRODCD          object
NPR_PREMIUM        float64
CLF_LIFECD           int64
NSP_SUBPROPOSAL      int64
NPR_SUMASSURED     float64
NLO_TYPE            object
NLO_AMOUNT         float64
OCCUPATION          object
AMOUNTPAID         float64
DATEPAID            object
PREMIUMDUEDATE      object
dtype: object

In [187]:
train['DATEPAID'] = pd.to_datetime(train['DATEPAID'], format='%Y-%m-%d')
test['DATEPAID'] = pd.to_datetime(test['DATEPAID'], format='%Y-%m-%d')

In [188]:
test['dayofweek'] = pd.DatetimeIndex(test['DATEPAID']).dayofweek
test['is_weekend'] = test.dayofweek.isin([5,6])*1
test['quarter'] = test['DATEPAID'].dt.quarter
test['month'] = test['DATEPAID'].dt.month
test['year'] = test['DATEPAID'].dt.year
test['day'] = test['DATEPAID'].dt.day
test['is_year_start'] = test['DATEPAID'].dt.is_year_start
test['is_year_end'] = test['DATEPAID'].dt.is_year_end
test['is_month_start'] = test['DATEPAID'].dt.is_month_start
test['is_month_end'] = test['DATEPAID'].dt.is_month_end
test['is_quarter_start'] = test['DATEPAID'].dt.is_quarter_start

In [189]:
train['dayofweek'] = pd.DatetimeIndex(train['DATEPAID']).dayofweek
train['is_weekend'] = train.dayofweek.isin([5,6])*1
train['quarter'] = train['DATEPAID'].dt.quarter
train['month'] = train['DATEPAID'].dt.month
train['year'] = train['DATEPAID'].dt.year
train['day'] = train['DATEPAID'].dt.day
train['is_year_start'] = train['DATEPAID'].dt.is_year_start
train['is_year_end'] = train['DATEPAID'].dt.is_year_end
train['is_month_start'] = train['DATEPAID'].dt.is_month_start
train['is_month_end'] = train['DATEPAID'].dt.is_month_end
train['is_quarter_start'] = train['DATEPAID'].dt.is_quarter_start

In [190]:
train['month'].fillna(method='bfill', inplace=True)
test['month'].fillna(method='bfill', inplace=True)
train['day'].fillna(method='ffill', inplace=True)
test['day'].fillna(method='ffill', inplace=True)
train['year'].fillna(2019, inplace=True)
test['year'].fillna(2019, inplace=True)

train['is_weekend'].fillna(method='ffill', inplace=True)
train['dayofweek'].fillna(method='ffill', inplace=True)
train['quarter'].fillna(method='bfill', inplace=True)
train['is_year_start'].fillna(method='ffill', inplace=True) 
train['is_year_end'].fillna(method='ffill', inplace=True)
train['is_month_start'].fillna(method='bfill', inplace=True)
train['is_month_end'].fillna(method='bfill', inplace=True)
train['is_quarter_start'].fillna(method='bfill', inplace=True)
test['is_weekend'].fillna(method='ffill', inplace=True)
test['quarter'].fillna(method='bfill', inplace=True)
test['is_year_start'].fillna(method='ffill', inplace=True) 
test['is_year_end'].fillna(method='ffill', inplace=True)
test['is_month_start'].fillna(method='bfill', inplace=True)
test['is_month_end'].fillna(method='bfill', inplace=True)
test['is_quarter_start'].fillna(method='bfill', inplace=True)
train['dayofweek'].fillna(method='ffill', inplace=True)

In [191]:
train['year'].unique()

array([2018., 2019., 2017.])

In [192]:
train.head()

,Policy ID,Lapse,NP2_EFFECTDATE,PPR_PRODCD,NPR_PREMIUM,CLF_LIFECD,NSP_SUBPROPOSAL,NPR_SUMASSURED,NLO_TYPE,NLO_AMOUNT,...,is_weekend,quarter,month,year,day,is_year_start,is_year_end,is_month_start,is_month_end,is_quarter_start
0,PID_MFAAYNJ,1.0,1/10/2018,PPR_PRODCD_KOFUYNN,1074.485839,4,333,27631.086529,NLO_TYPE_DPBHSAH,NaN,...,0,1.0,1.0,2018.0,10.0,False,False,False,False,False
1,PID_TICDPAY,1.0,1/4/2017,PPR_PRODCD_165U4LY,1243.869117,1,111,145010.003453,NLO_TYPE_XAJI0Y6,215.354032,...,0,2.0,6.0,2019.0,10.0,False,False,False,False,False
2,PID_SPACC3N,1.0,1/3/2018,PPR_PRODCD_KOFUYNN,193.905482,6,333,45513.429325,NLO_TYPE_DPBHSAH,NaN,...,0,2.0,6.0,2018.0,13.0,False,False,False,False,False
3,PID_HEW7QK9,1.0,1/4/2018,PPR_PRODCD_B2KVCE7,191.160414,3,333,NaN,NLO_TYPE_DPBHSAH,NaN,...,0,2.0,4.0,2018.0,4.0,False,False,False,False,False
4,PID_1TY2H0B,1.0,1/6/2017,PPR_PRODCD_165U4LY,865.062596,2,222,100384.546919,NLO_TYPE_XAJI0Y6,215.354032,...,1,1.0,3.0,2017.0,11.0,False,False,False,False,False


In [193]:
train.isna().sum()

Policy ID               0
Lapse               43707
NP2_EFFECTDATE          0
PPR_PRODCD              0
NPR_PREMIUM             3
CLF_LIFECD              0
NSP_SUBPROPOSAL         0
NPR_SUMASSURED       9388
NLO_TYPE                0
NLO_AMOUNT          18676
OCCUPATION              0
AMOUNTPAID          19705
DATEPAID            19462
PREMIUMDUEDATE      19883
dayofweek               0
is_weekend              0
quarter                 0
month                   0
year                    0
day                     0
is_year_start           0
is_year_end             0
is_month_start          0
is_month_end            0
is_quarter_start        0
dtype: int64

In [196]:
test.head()

,Policy ID,Lapse,NP2_EFFECTDATE,PPR_PRODCD,NPR_PREMIUM,CLF_LIFECD,NSP_SUBPROPOSAL,NPR_SUMASSURED,NLO_TYPE,NLO_AMOUNT,...,is_weekend,quarter,month,year,day,is_year_start,is_year_end,is_month_start,is_month_end,is_quarter_start
0,PID_4928TWH,0.0,1/8/2017,PPR_PRODCD_XRWHUER,42911.077278,1,111,340667.737114,NLO_TYPE_XAJI0Y6,16702.717882,...,0,3.0,7.0,2017.0,31.0,False,False,False,True,False
1,PID_KBLLEGK,0.0,1/5/2018,PPR_PRODCD_64QNIHM,3561.268991,1,111,165053.885026,NLO_TYPE_XAJI0Y6,600.259636,...,0,2.0,4.0,2018.0,30.0,False,False,False,True,False
2,PID_90F0QA3,0.0,1/9/2019,PPR_PRODCD_64QNIHM,6164.812836,1,111,252467.025723,NLO_TYPE_XTHV3A3,1343.332956,...,0,4.0,12.0,2019.0,30.0,False,False,False,False,False
3,PID_18F3NHF,0.0,1/12/2019,PPR_PRODCD_GLE2MHV,2278.189789,1,111,238857.872515,NLO_TYPE_XAJI0Y6,1318.742708,...,0,4.0,12.0,2019.0,30.0,False,False,False,False,False
4,PID_SX4QUVO,0.0,1/3/2019,PPR_PRODCD_W0F6GK1,1619.046308,1,111,145010.003453,NLO_TYPE_XAJI0Y6,332.816358,...,0,4.0,12.0,2019.0,30.0,False,False,False,False,False


In [204]:
train[train['year'] == 2019]

,Policy ID,Lapse,NP2_EFFECTDATE,PPR_PRODCD,NPR_PREMIUM,CLF_LIFECD,NSP_SUBPROPOSAL,NPR_SUMASSURED,NLO_TYPE,NLO_AMOUNT,...,is_weekend,quarter,month,year,day,is_year_start,is_year_end,is_month_start,is_month_end,is_quarter_start
1,PID_TICDPAY,1.0,1/4/2017,PPR_PRODCD_165U4LY,1243.869117,1,111,145010.003453,NLO_TYPE_XAJI0Y6,215.354032,...,0,2.0,6.0,2019.0,10.0,False,False,False,False,False
27,PID_4RIHXCB,1.0,1/2/2019,PPR_PRODCD_KOFUYNN,193.905482,6,333,45513.429325,NLO_TYPE_DPBHSAH,NaN,...,0,1.0,2.0,2019.0,18.0,False,False,False,False,False
28,PID_E4KB06X,1.0,1/5/2019,PPR_PRODCD_KOFUYNN,359.045970,3,555,74968.903115,NLO_TYPE_DPBHSAH,NaN,...,0,1.0,2.0,2019.0,18.0,False,False,False,False,False
31,PID_AXQLOL2,1.0,1/1/2019,PPR_PRODCD_KOFUYNN,217.975890,3,333,45513.429325,NLO_TYPE_DPBHSAH,NaN,...,0,3.0,8.0,2019.0,3.0,False,False,False,False,False
35,PID_QM99DX4,1.0,1/5/2019,PPR_PRODCD_KOFUYNN,217.975890,3,222,45513.429325,NLO_TYPE_DPBHSAH,NaN,...,0,2.0,6.0,2019.0,31.0,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51498,PID_CO1GI13,NaN,1/5/2019,PPR_PRODCD_W0F6GK1,1362.623244,1,111,145010.003453,NLO_TYPE_XAJI0Y6,280.105210,...,0,2.0,4.0,2019.0,29.0,False,False,False,False,False
51513,PID_9NTT4KR,NaN,1/5/2019,PPR_PRODCD_EJ7YKFV,4047.406097,1,111,238857.872515,NLO_TYPE_XAJI0Y6,1512.116285,...,0,1.0,2.0,2019.0,6.0,False,False,False,False,False
51542,PID_LILMX13,NaN,1/2/2019,PPR_PRODCD_165U4LY,401.191521,1,111,45513.429325,NLO_TYPE_XAJI0Y6,217.975890,...,0,3.0,8.0,2019.0,2.0,False,False,False,False,False
51592,PID_0NMSRAK,NaN,1/11/2017,PPR_PRODCD_165U4LY,401.191521,1,111,45513.429325,NLO_TYPE_XAJI0Y6,217.975890,...,0,1.0,1.0,2019.0,9.0,False,False,False,False,False


In [205]:
train[train['Policy ID'] == 'PID_KELHNU0']

,Policy ID,Lapse,NP2_EFFECTDATE,PPR_PRODCD,NPR_PREMIUM,CLF_LIFECD,NSP_SUBPROPOSAL,NPR_SUMASSURED,NLO_TYPE,NLO_AMOUNT,...,is_weekend,quarter,month,year,day,is_year_start,is_year_end,is_month_start,is_month_end,is_quarter_start
51617,PID_KELHNU0,NaN,1/5/2017,PPR_PRODCD_W0F6GK1,1625.038466,1,111,145010.003453,NLO_TYPE_XAJI0Y6,335.032269,...,0,2.0,5.0,2019.0,30.0,False,False,False,False,False


In [212]:
train['Lapse'].fillna(0, inplace=True)

In [213]:
train['Lapse'].value_counts()

0.0    43707
1.0     7978
Name: Lapse, dtype: int64

In [214]:
train.isna().sum()

Policy ID               0
Lapse                   0
NP2_EFFECTDATE          0
PPR_PRODCD              0
NPR_PREMIUM             3
CLF_LIFECD              0
NSP_SUBPROPOSAL         0
NPR_SUMASSURED       9388
NLO_TYPE                0
NLO_AMOUNT          18676
OCCUPATION              0
AMOUNTPAID          19705
DATEPAID            19462
PREMIUMDUEDATE      19883
dayofweek               0
is_weekend              0
quarter                 0
month                   0
year                    0
day                     0
is_year_start           0
is_year_end             0
is_month_start          0
is_month_end            0
is_quarter_start        0
dtype: int64

In [215]:
#train['PPR_PRODCD+NLO'] = train['PPR_PRODCD'] +'_'+ train['NLO_TYPE']
#train['OCC+NLO'] = train['OCCUPATION'] +'_'+ train['NLO_TYPE']
#test['PPR_PRODCD+NLO'] = test['PPR_PRODCD'] +'_'+ test['NLO_TYPE']
#test['OCC+NLO'] = test['OCCUPATION'] +'_'+ test['NLO_TYPE']

In [216]:
print(test.shape,train.shape)

(43707, 25) (51685, 25)


In [217]:
train.fillna(-1, inplace=True)
test.fillna(-1, inplace=True)

In [218]:
train=train.drop(['NP2_EFFECTDATE','PREMIUMDUEDATE','DATEPAID'], axis=1)
test=test.drop(['NP2_EFFECTDATE','PREMIUMDUEDATE','DATEPAID','Lapse'], axis=1)

In [219]:
x=train.drop(['Lapse'], axis=1)
y=train.Lapse

In [220]:
test.columns

Index(['Policy ID', 'PPR_PRODCD', 'NPR_PREMIUM', 'CLF_LIFECD',
       'NSP_SUBPROPOSAL', 'NPR_SUMASSURED', 'NLO_TYPE', 'NLO_AMOUNT',
       'OCCUPATION', 'AMOUNTPAID', 'dayofweek', 'is_weekend', 'quarter',
       'month', 'year', 'day', 'is_year_start', 'is_year_end',
       'is_month_start', 'is_month_end', 'is_quarter_start'],
      dtype='object')

In [221]:
train.columns

Index(['Policy ID', 'Lapse', 'PPR_PRODCD', 'NPR_PREMIUM', 'CLF_LIFECD',
       'NSP_SUBPROPOSAL', 'NPR_SUMASSURED', 'NLO_TYPE', 'NLO_AMOUNT',
       'OCCUPATION', 'AMOUNTPAID', 'dayofweek', 'is_weekend', 'quarter',
       'month', 'year', 'day', 'is_year_start', 'is_year_end',
       'is_month_start', 'is_month_end', 'is_quarter_start'],
      dtype='object')

In [224]:
x.shape

(51685, 21)

In [225]:
cate_features_index=np.where(x.dtypes != float) [0]; cate_features_index

array([ 0,  1,  3,  4,  6,  8, 11, 16, 17, 18, 19, 20], dtype=int64)

In [226]:
from sklearn.model_selection import train_test_split, KFold, cross_validate, cross_val_score
from catboost import Pool, CatBoostClassifier, cv#0.035130

In [250]:
xtrain,xtest,ytrain,ytest = train_test_split(x,y,test_size=0.15, random_state=12345)#best depth 3, no new features bynas
model = CatBoostClassifier(iterations=1000, depth=4, learning_rate=0.03,eval_metric='Logloss',use_best_model=True, random_seed=12345)
model.fit(xtrain,ytrain,cat_features=cate_features_index,eval_set=(xtest,ytest))

0:	learn: 0.6480134	test: 0.6485160	best: 0.6485160 (0)	total: 116ms	remaining: 3m 51s
1:	learn: 0.6115871	test: 0.6123053	best: 0.6123053 (1)	total: 203ms	remaining: 3m 22s
2:	learn: 0.5779079	test: 0.5788678	best: 0.5788678 (2)	total: 268ms	remaining: 2m 58s
3:	learn: 0.5493818	test: 0.5507202	best: 0.5507202 (3)	total: 322ms	remaining: 2m 40s
4:	learn: 0.5236169	test: 0.5252930	best: 0.5252930 (4)	total: 438ms	remaining: 2m 54s
5:	learn: 0.5013094	test: 0.5028972	best: 0.5028972 (5)	total: 529ms	remaining: 2m 55s
6:	learn: 0.4886983	test: 0.4907329	best: 0.4907329 (6)	total: 646ms	remaining: 3m 3s
7:	learn: 0.4661375	test: 0.4684026	best: 0.4684026 (7)	total: 699ms	remaining: 2m 54s
8:	learn: 0.4506872	test: 0.4531802	best: 0.4531802 (8)	total: 809ms	remaining: 2m 59s
9:	learn: 0.4396788	test: 0.4422911	best: 0.4422911 (9)	total: 898ms	remaining: 2m 58s
10:	learn: 0.4324945	test: 0.4353256	best: 0.4353256 (10)	total: 942ms	remaining: 2m 50s
11:	learn: 0.4173959	test: 0.4203160	best:

94:	learn: 0.3033680	test: 0.3113678	best: 0.3113678 (94)	total: 7.42s	remaining: 2m 28s
95:	learn: 0.3032817	test: 0.3113051	best: 0.3113051 (95)	total: 7.5s	remaining: 2m 28s
96:	learn: 0.3032103	test: 0.3112510	best: 0.3112510 (96)	total: 7.58s	remaining: 2m 28s
97:	learn: 0.3031461	test: 0.3112208	best: 0.3112208 (97)	total: 7.67s	remaining: 2m 28s
98:	learn: 0.3030146	test: 0.3110852	best: 0.3110852 (98)	total: 7.73s	remaining: 2m 28s
99:	learn: 0.3029386	test: 0.3110258	best: 0.3110258 (99)	total: 7.8s	remaining: 2m 28s
100:	learn: 0.3028526	test: 0.3109677	best: 0.3109677 (100)	total: 7.86s	remaining: 2m 27s
101:	learn: 0.3027277	test: 0.3108548	best: 0.3108548 (101)	total: 7.96s	remaining: 2m 28s
102:	learn: 0.3025859	test: 0.3107590	best: 0.3107590 (102)	total: 8.02s	remaining: 2m 27s
103:	learn: 0.3025153	test: 0.3106984	best: 0.3106984 (103)	total: 8.1s	remaining: 2m 27s
104:	learn: 0.3024515	test: 0.3106463	best: 0.3106463 (104)	total: 8.18s	remaining: 2m 27s
105:	learn: 0.

186:	learn: 0.2978056	test: 0.3071772	best: 0.3071772 (186)	total: 14.8s	remaining: 2m 23s
187:	learn: 0.2977849	test: 0.3071488	best: 0.3071488 (187)	total: 14.9s	remaining: 2m 23s
188:	learn: 0.2977116	test: 0.3070650	best: 0.3070650 (188)	total: 15s	remaining: 2m 23s
189:	learn: 0.2976835	test: 0.3070583	best: 0.3070583 (189)	total: 15.1s	remaining: 2m 23s
190:	learn: 0.2976268	test: 0.3070073	best: 0.3070073 (190)	total: 15.1s	remaining: 2m 23s
191:	learn: 0.2976077	test: 0.3070008	best: 0.3070008 (191)	total: 15.2s	remaining: 2m 23s
192:	learn: 0.2975795	test: 0.3069861	best: 0.3069861 (192)	total: 15.3s	remaining: 2m 23s
193:	learn: 0.2975510	test: 0.3069788	best: 0.3069788 (193)	total: 15.4s	remaining: 2m 23s
194:	learn: 0.2975221	test: 0.3069509	best: 0.3069509 (194)	total: 15.4s	remaining: 2m 22s
195:	learn: 0.2974781	test: 0.3069224	best: 0.3069224 (195)	total: 15.5s	remaining: 2m 22s
196:	learn: 0.2974676	test: 0.3069230	best: 0.3069224 (195)	total: 15.6s	remaining: 2m 22s
1

278:	learn: 0.2950254	test: 0.3052104	best: 0.3052104 (278)	total: 22.3s	remaining: 2m 17s
279:	learn: 0.2950023	test: 0.3051992	best: 0.3051992 (279)	total: 22.3s	remaining: 2m 17s
280:	learn: 0.2949559	test: 0.3051879	best: 0.3051879 (280)	total: 22.4s	remaining: 2m 16s
281:	learn: 0.2949381	test: 0.3051716	best: 0.3051716 (281)	total: 22.5s	remaining: 2m 16s
282:	learn: 0.2949138	test: 0.3051351	best: 0.3051351 (282)	total: 22.5s	remaining: 2m 16s
283:	learn: 0.2948387	test: 0.3050996	best: 0.3050996 (283)	total: 22.6s	remaining: 2m 16s
284:	learn: 0.2948270	test: 0.3050979	best: 0.3050979 (284)	total: 22.7s	remaining: 2m 16s
285:	learn: 0.2947580	test: 0.3050279	best: 0.3050279 (285)	total: 22.7s	remaining: 2m 16s
286:	learn: 0.2947203	test: 0.3050026	best: 0.3050026 (286)	total: 22.8s	remaining: 2m 16s
287:	learn: 0.2947076	test: 0.3050013	best: 0.3050013 (287)	total: 22.9s	remaining: 2m 16s
288:	learn: 0.2946952	test: 0.3049851	best: 0.3049851 (288)	total: 22.9s	remaining: 2m 15s

371:	learn: 0.2925122	test: 0.3036470	best: 0.3036470 (371)	total: 29.4s	remaining: 2m 8s
372:	learn: 0.2924884	test: 0.3036308	best: 0.3036308 (372)	total: 29.5s	remaining: 2m 8s
373:	learn: 0.2924646	test: 0.3036092	best: 0.3036092 (373)	total: 29.6s	remaining: 2m 8s
374:	learn: 0.2924218	test: 0.3035517	best: 0.3035517 (374)	total: 29.7s	remaining: 2m 8s
375:	learn: 0.2924046	test: 0.3035276	best: 0.3035276 (375)	total: 29.8s	remaining: 2m 8s
376:	learn: 0.2923861	test: 0.3035108	best: 0.3035108 (376)	total: 29.8s	remaining: 2m 8s
377:	learn: 0.2923803	test: 0.3035134	best: 0.3035108 (376)	total: 29.9s	remaining: 2m 8s
378:	learn: 0.2923325	test: 0.3034714	best: 0.3034714 (378)	total: 30s	remaining: 2m 8s
379:	learn: 0.2922821	test: 0.3034340	best: 0.3034340 (379)	total: 30.1s	remaining: 2m 8s
380:	learn: 0.2922587	test: 0.3034093	best: 0.3034093 (380)	total: 30.1s	remaining: 2m 7s
381:	learn: 0.2922531	test: 0.3034083	best: 0.3034083 (381)	total: 30.2s	remaining: 2m 7s
382:	learn: 

...

1550:	learn: 0.2787522	test: 0.2982372	best: 0.2982372 (1550)	total: 2m 10s	remaining: 37.7s
1551:	learn: 0.2787498	test: 0.2982369	best: 0.2982369 (1551)	total: 2m 10s	remaining: 37.6s
1552:	learn: 0.2787412	test: 0.2982338	best: 0.2982338 (1552)	total: 2m 10s	remaining: 37.5s
1553:	learn: 0.2787152	test: 0.2982279	best: 0.2982279 (1553)	total: 2m 10s	remaining: 37.4s
1554:	learn: 0.2786855	test: 0.2981998	best: 0.2981998 (1554)	total: 2m 10s	remaining: 37.4s
1555:	learn: 0.2786776	test: 0.2981963	best: 0.2981963 (1555)	total: 2m 10s	remaining: 37.3s
1556:	learn: 0.2786650	test: 0.2981957	best: 0.2981957 (1556)	total: 2m 10s	remaining: 37.2s
1557:	learn: 0.2786566	test: 0.2982014	best: 0.2981957 (1556)	total: 2m 10s	remaining: 37.1s
1558:	learn: 0.2786474	test: 0.2982037	best: 0.2981957 (1556)	total: 2m 10s	remaining: 37s
1559:	learn: 0.2786427	test: 0.2982053	best: 0.2981957 (1556)	total: 2m 10s	remaining: 36.9s
1560:	learn: 0.2786382	test: 0.2982084	best: 0.2981957 (1556)	total: 2m 

1639:	learn: 0.2781365	test: 0.2980271	best: 0.2980271 (1639)	total: 2m 17s	remaining: 30.3s
1640:	learn: 0.2781341	test: 0.2980243	best: 0.2980243 (1640)	total: 2m 18s	remaining: 30.2s
1641:	learn: 0.2781294	test: 0.2980271	best: 0.2980243 (1640)	total: 2m 18s	remaining: 30.1s
1642:	learn: 0.2781245	test: 0.2980375	best: 0.2980243 (1640)	total: 2m 18s	remaining: 30s
1643:	learn: 0.2781211	test: 0.2980374	best: 0.2980243 (1640)	total: 2m 18s	remaining: 30s
1644:	learn: 0.2781143	test: 0.2980471	best: 0.2980243 (1640)	total: 2m 18s	remaining: 29.9s
1645:	learn: 0.2781041	test: 0.2980390	best: 0.2980243 (1640)	total: 2m 18s	remaining: 29.8s
1646:	learn: 0.2780934	test: 0.2980456	best: 0.2980243 (1640)	total: 2m 18s	remaining: 29.7s
1647:	learn: 0.2780749	test: 0.2980263	best: 0.2980243 (1640)	total: 2m 18s	remaining: 29.6s
1648:	learn: 0.2780654	test: 0.2980308	best: 0.2980243 (1640)	total: 2m 18s	remaining: 29.6s
1649:	learn: 0.2780605	test: 0.2980309	best: 0.2980243 (1640)	total: 2m 18

1729:	learn: 0.2775549	test: 0.2980250	best: 0.2980030 (1669)	total: 2m 26s	remaining: 22.8s
1730:	learn: 0.2775502	test: 0.2980276	best: 0.2980030 (1669)	total: 2m 26s	remaining: 22.7s
1731:	learn: 0.2775428	test: 0.2980260	best: 0.2980030 (1669)	total: 2m 26s	remaining: 22.7s
1732:	learn: 0.2775410	test: 0.2980231	best: 0.2980030 (1669)	total: 2m 26s	remaining: 22.6s
1733:	learn: 0.2775384	test: 0.2980239	best: 0.2980030 (1669)	total: 2m 26s	remaining: 22.5s
1734:	learn: 0.2775377	test: 0.2980237	best: 0.2980030 (1669)	total: 2m 26s	remaining: 22.4s
1735:	learn: 0.2775323	test: 0.2980277	best: 0.2980030 (1669)	total: 2m 26s	remaining: 22.3s
1736:	learn: 0.2775259	test: 0.2980256	best: 0.2980030 (1669)	total: 2m 26s	remaining: 22.2s
1737:	learn: 0.2775217	test: 0.2980255	best: 0.2980030 (1669)	total: 2m 26s	remaining: 22.1s
1738:	learn: 0.2775163	test: 0.2980380	best: 0.2980030 (1669)	total: 2m 26s	remaining: 22.1s
1739:	learn: 0.2775128	test: 0.2980332	best: 0.2980030 (1669)	total: 2

1818:	learn: 0.2768702	test: 0.2978723	best: 0.2978723 (1818)	total: 2m 34s	remaining: 15.3s
1819:	learn: 0.2768625	test: 0.2978699	best: 0.2978699 (1819)	total: 2m 34s	remaining: 15.2s
1820:	learn: 0.2768532	test: 0.2978758	best: 0.2978699 (1819)	total: 2m 34s	remaining: 15.2s
1821:	learn: 0.2768496	test: 0.2978762	best: 0.2978699 (1819)	total: 2m 34s	remaining: 15.1s
1822:	learn: 0.2768417	test: 0.2978790	best: 0.2978699 (1819)	total: 2m 34s	remaining: 15s
1823:	learn: 0.2768391	test: 0.2978782	best: 0.2978699 (1819)	total: 2m 34s	remaining: 14.9s
1824:	learn: 0.2768305	test: 0.2978833	best: 0.2978699 (1819)	total: 2m 34s	remaining: 14.8s
1825:	learn: 0.2768196	test: 0.2978725	best: 0.2978699 (1819)	total: 2m 34s	remaining: 14.7s
1826:	learn: 0.2768147	test: 0.2978801	best: 0.2978699 (1819)	total: 2m 34s	remaining: 14.6s
1827:	learn: 0.2768117	test: 0.2978801	best: 0.2978699 (1819)	total: 2m 34s	remaining: 14.6s
1828:	learn: 0.2768080	test: 0.2978743	best: 0.2978699 (1819)	total: 2m 

1909:	learn: 0.2762890	test: 0.2978555	best: 0.2978484 (1894)	total: 2m 42s	remaining: 7.66s
1910:	learn: 0.2762861	test: 0.2978605	best: 0.2978484 (1894)	total: 2m 42s	remaining: 7.58s
1911:	learn: 0.2762839	test: 0.2978612	best: 0.2978484 (1894)	total: 2m 42s	remaining: 7.49s
1912:	learn: 0.2762777	test: 0.2978638	best: 0.2978484 (1894)	total: 2m 42s	remaining: 7.41s
1913:	learn: 0.2762700	test: 0.2978614	best: 0.2978484 (1894)	total: 2m 43s	remaining: 7.33s
1914:	learn: 0.2762660	test: 0.2978622	best: 0.2978484 (1894)	total: 2m 43s	remaining: 7.24s
1915:	learn: 0.2762454	test: 0.2978617	best: 0.2978484 (1894)	total: 2m 43s	remaining: 7.16s
1916:	learn: 0.2762382	test: 0.2978632	best: 0.2978484 (1894)	total: 2m 43s	remaining: 7.08s
1917:	learn: 0.2762354	test: 0.2978644	best: 0.2978484 (1894)	total: 2m 43s	remaining: 6.99s
1918:	learn: 0.2762182	test: 0.2978578	best: 0.2978484 (1894)	total: 2m 43s	remaining: 6.91s
1919:	learn: 0.2762173	test: 0.2978587	best: 0.2978484 (1894)	total: 2

In [246]:
pred =model.predict_proba(test)[:,1]

In [247]:
d = {"Policy ID": test['Policy ID'], 'Lapse':pred}
train_predictions = pd.DataFrame(data=d)
train_predictions = train_predictions[["Policy ID", 'Lapse']]

In [248]:
train_predictions.head()

,Policy ID,Lapse
0,PID_4928TWH,0.574580
1,PID_KBLLEGK,0.000314
2,PID_90F0QA3,0.000109
3,PID_18F3NHF,0.005054
4,PID_SX4QUVO,0.098095


In [249]:
train_predictions.to_csv('newapproach.csv', index=False)

In [100]:
a =pd.read_csv('final1.csv')
b =pd.read_csv('train001.csv')

In [101]:
a['Lapse'] =a['Lapse']*0.60 +b['Lapse']*0.40

In [102]:
a.to_csv('ensemble1.csv', index=False) 

In [103]:
a.head()

,Policy ID,Lapse
0,PID_4928TWH,0.741894
1,PID_KBLLEGK,0.000212
2,PID_90F0QA3,0.000057
3,PID_18F3NHF,0.003310
4,PID_SX4QUVO,0.057929


In [96]:
for col in ['Policy ID']:
    data_col = data[col].unique()
    sample_col = sample[col].unique()
    
    print ('Not in sample ' + col + ' ',[i  for i in data_col if i not in data_col])
    print ('Not in data ' + col + ' ',[i  for i in sample_col  if i not in sample_col])
    print('==' * 10)

Not in sample Policy ID  []
Not in data Policy ID  []
